In [13]:
import requests 
import random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from time import sleep

In [14]:
# Con esto podemos evitar que el navegador detecte 
#que estamos usando un software de automatización
options = Options()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

In [15]:
# Podemos guardar las cookies

options.add_argument('user-data-dir=/tmp/selenium_profile') 

In [16]:
from fake_useragent import UserAgent

user = UserAgent().random

print(user)

Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0


In [17]:
options.add_argument(f'user-agent={user}')

In [29]:
#estadisticas para los equipos temporada 23/24
def team_tables_23_24(driver, league):
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text("Leagues")
    #hover over element to chose the specific league
    a.move_to_element(m).perform()
    #identify sub menu element (one of the 5 leagues)
    n = driver.find_element_by_link_text(league)
    # hover over element and click (one of the 5 leagues)
    a.move_to_element(n).click().perform()
    print("Page title: " + driver.title)
    
    league_name = league.replace(" ", "_")
    
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    league_name = league.replace(" ", "_")
    
    table_data = []
    lista_teams = []
    
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')]
    colnames = [cols[i] for i in [2,3,4,5,6,7,8,9,10,12,13,14,15,-1]]
    
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.team.borderRightContent')]
    link = [i.find_element(By.TAG_NAME, 'a').get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'td.team.borderRightContent')]


    mp = [i.text for i in driver.find_elements(By.CLASS_NAME, 'mp') if i.text.isdigit()]
    win = [i.text for i in driver.find_elements(By.CLASS_NAME, 'win') if i.text.isdigit()]
    draw = [i.text for i in driver.find_elements(By.CLASS_NAME, 'draw') if i.text.isdigit()]
    loss = [i.text for i in driver.find_elements(By.CLASS_NAME, 'loss') if i.text.isdigit()]
    gf = [i.text for i in driver.find_elements(By.CLASS_NAME, 'gf') if i.text.isdigit()]
    ga = [i.text for i in driver.find_elements(By.CLASS_NAME, 'ga') if i.text.isdigit()]
    gd = [i.text for i in driver.find_elements(By.CLASS_NAME, 'gd')[1:]]
    pts = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.points.bold') if i.text.isdigit()]
    ppg = [i.text for i in driver.find_elements(By.CLASS_NAME, 'ppg')[1:]]
    cs = [i.text for i in driver.find_elements(By.CLASS_NAME, 'cs')[1:]]
    btts = [i.text for i in driver.find_elements(By.CLASS_NAME, 'btts')[1:]]
    fts = [i.text for i in driver.find_elements(By.CLASS_NAME, 'fts')[1:]]
    avgg = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.avg.bold')]
    
    
    table_data.append([team, mp, win, draw, loss, gf, ga, gd, pts, ppg, cs, btts, fts, avgg]) 
    table = {col : data for col, data in zip(colnames, table_data[0])}
    league_df = pd.DataFrame(table)
    return league_df


In [30]:
# guardar tablas de las ligas para temporada 23/24
def save_table_pd_23_24(league_df:pd.DataFrame, league:str)->None:

    league_df.to_csv(f'{league}_league_23_24.csv', index=False)

In [22]:
chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://footystats.org/")
team_tables_23_24(driver, "Bundesliga")

Page title: Bundesliga 2023/24 Table & Stats | FootyStats


,Team,MP,W,D,L,GF,GA,GD,Pts,PPG,CS,BTTS,xGF,AVG
0,Bayer 04 Leverkusen,9,8,1,0,27,8,+19,25,2.78,33%,67%,2.02,3.89
1,FC Bayern München,9,7,2,0,34,7,+27,23,2.56,44%,56%,2.42,4.56
2,VfB Stuttgart 1893,9,7,0,2,27,11,+16,21,2.33,44%,56%,1.81,4.22
3,BVB 09 Borussia Dortmund,9,6,3,0,20,11,+9,21,2.33,33%,67%,1.67,3.44
4,Rasen Ballsport Leipzig,9,6,2,1,25,7,+18,20,2.22,56%,44%,1.62,3.56
5,TSG 1899 Hoffenheim,9,6,0,3,20,16,+4,18,2.00,11%,89%,1.39,4.00
6,Eintracht Frankfurt,9,3,5,1,12,9,+3,14,1.56,33%,56%,0.99,2.33
7,SC Freiburg,9,4,1,4,10,16,-6,13,1.44,33%,44%,1.15,2.89
8,VfL Wolfsburg,9,4,0,5,13,14,-1,12,1.33,22%,67%,1.44,3.00
9,FC Augsburg,9,3,2,4,18,21,-3,11,1.22,0%,78%,1.18,4.33


In [31]:
#temporada 23/24
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://footystats.org/")

leagues = ["Premier League", "Serie A", "Ligue 1", "La Liga", "Bundesliga"]

errores = 0
for l in leagues:
    try:
        league_df = team_tables_23_24(driver, l)
        save_table_pd_23_24(league_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1
    
driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League 2023/24 Table & Stats | FootyStats
Page title: Serie A 2023/24 Table & Stats | FootyStats
Page title: Ligue 1 2023/24 Table & Stats | FootyStats
Page title: La Liga 2023/24 Table & Stats | FootyStats
Page title: Bundesliga 2023/24 Table & Stats | FootyStats
Proceso finalizado en 0:01:36.549477 con 0 errores


In [39]:
#estadisticas de los equipos para las temporadas 22/23
def team_data_22_23(driver, league)->pd.DataFrame:
#object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text("Leagues")
    #hover over element to chose the specific league
    a.move_to_element(m).perform()
    #identify sub menu element (one of the 5 leagues)
    n = driver.find_element_by_link_text(league)
    # hover over element and click (one of the 5 leagues)
    a.move_to_element(n).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random de dormir entre 6 y 10 segundos
    sleep(random.uniform(4.0, 10.0))
    #identify element: change to a different season
    a = ActionChains(driver)
    y = driver.find_element_by_css_selector("div.drop-down-parent.fl.boldFont")
    #hover over element
    a.move_to_element(y).perform()
    #identify sub menu element
    y1 = driver.find_element_by_link_text("2022/23")
    # hover over element and click
    a.move_to_element(y1).click().perform()
    sleep(random.uniform(6.0, 10.0))
    print("Page title: " + driver.title)

    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    league_name = league.replace(" ", "_")
    
    sleep(random.uniform(4.0, 10.0))
    
    table_data = []
    lista_teams = []
    
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')]
    colnames = [cols[i] for i in [2,3,4,5,6,7,8,9,10,12,13,14,15,-1]]
    
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.team.borderRightContent')]
    link = [i.find_element(By.TAG_NAME, 'a').get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'td.team.borderRightContent')]


    mp = [i.text for i in driver.find_elements(By.CLASS_NAME, 'mp') if i.text.isdigit()]
    win = [i.text for i in driver.find_elements(By.CLASS_NAME, 'win') if i.text.isdigit()]
    draw = [i.text for i in driver.find_elements(By.CLASS_NAME, 'draw') if i.text.isdigit()]
    loss = [i.text for i in driver.find_elements(By.CLASS_NAME, 'loss') if i.text.isdigit()]
    gf = [i.text for i in driver.find_elements(By.CLASS_NAME, 'gf') if i.text.isdigit()]
    ga = [i.text for i in driver.find_elements(By.CLASS_NAME, 'ga') if i.text.isdigit()]
    gd = [i.text for i in driver.find_elements(By.CLASS_NAME, 'gd')[1:]]
    pts = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.points.bold') if i.text.isdigit()]
    ppg = [i.text for i in driver.find_elements(By.CLASS_NAME, 'ppg')[1:]]
    cs = [i.text for i in driver.find_elements(By.CLASS_NAME, 'cs')[1:]]
    btts = [i.text for i in driver.find_elements(By.CLASS_NAME, 'btts')[1:]]
    fts = [i.text for i in driver.find_elements(By.CLASS_NAME, 'fts')[1:]]
    avgg = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.avg.bold')]
    
    
    lista_teams.append([team, link])
    
    
    table_data.append([team, mp, win, draw, loss, gf, ga, gd, pts, ppg, cs, btts, fts, avgg]) 
    table = {col : data for col, data in zip(colnames, table_data[0])}
    league_df = pd.DataFrame(table)
    return league_df



In [40]:
# guardar tablas de las ligas para temporada 22/23
def save_table_pd_22_23(league_df:pd.DataFrame, league:str)->None:

    league_df.to_csv(f'{league}_league_22_23.csv', index=False)

In [41]:
#temporada 22/23
chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://footystats.org/")

leagues = ["Premier League", "Serie A", "Ligue 1", "La Liga", "Bundesliga"]

errores = 0
for l in leagues:
    try:
        league_df = team_data_22_23(driver, l)
        save_table_pd_22_23(league_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1
    
driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League 2023/24 Table & Stats | FootyStats
Page title: Premier League 2023/24 Table & Stats | FootyStats
Page title: Serie A 2023/24 Table & Stats | FootyStats
Page title: Serie A 2023/24 Table & Stats | FootyStats
Page title: Ligue 1 2023/24 Table & Stats | FootyStats
Page title: Ligue 1 2023/24 Table & Stats | FootyStats
Page title: La Liga 2023/24 Table & Stats | FootyStats
Page title: La Liga 2023/24 Table & Stats | FootyStats
Page title: Bundesliga 2023/24 Table & Stats | FootyStats
Page title: Bundesliga 2023/24 Table & Stats | FootyStats
Proceso finalizado en 0:11:22.273013 con 0 errores


In [42]:
#estadisticas de los equipos para las temporadas 21/22
def team_data_21_22(driver, league)->pd.DataFrame:
#object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text("Leagues")
    #hover over element to chose the specific league
    a.move_to_element(m).perform()
    #identify sub menu element (one of the 5 leagues)
    n = driver.find_element_by_link_text(league)
    # hover over element and click (one of the 5 leagues)
    a.move_to_element(n).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random de dormir entre 6 y 10 segundos
    sleep(random.uniform(4.0, 10.0))
    #identify element: change to a different season
    a = ActionChains(driver)
    y = driver.find_element_by_css_selector("div.drop-down-parent.fl.boldFont")
    #hover over element
    a.move_to_element(y).perform()
    #identify sub menu element
    y1 = driver.find_element_by_link_text("2021/22")
    # hover over element and click
    a.move_to_element(y1).click().perform()
    sleep(random.uniform(6.0, 10.0))
    print("Page title: " + driver.title)

    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    league_name = league.replace(" ", "_")
    
    sleep(random.uniform(4.0, 10.0))
    
    table_data = []
    lista_teams = []
    
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')]
    colnames = [cols[i] for i in [2,3,4,5,6,7,8,9,10,12,13,14,15,-1]]
    
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.team.borderRightContent')]
    link = [i.find_element(By.TAG_NAME, 'a').get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'td.team.borderRightContent')]


    mp = [i.text for i in driver.find_elements(By.CLASS_NAME, 'mp') if i.text.isdigit()]
    win = [i.text for i in driver.find_elements(By.CLASS_NAME, 'win') if i.text.isdigit()]
    draw = [i.text for i in driver.find_elements(By.CLASS_NAME, 'draw') if i.text.isdigit()]
    loss = [i.text for i in driver.find_elements(By.CLASS_NAME, 'loss') if i.text.isdigit()]
    gf = [i.text for i in driver.find_elements(By.CLASS_NAME, 'gf') if i.text.isdigit()]
    ga = [i.text for i in driver.find_elements(By.CLASS_NAME, 'ga') if i.text.isdigit()]
    gd = [i.text for i in driver.find_elements(By.CLASS_NAME, 'gd')[1:]]
    pts = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.points.bold') if i.text.isdigit()]
    ppg = [i.text for i in driver.find_elements(By.CLASS_NAME, 'ppg')[1:]]
    cs = [i.text for i in driver.find_elements(By.CLASS_NAME, 'cs')[1:]]
    btts = [i.text for i in driver.find_elements(By.CLASS_NAME, 'btts')[1:]]
    fts = [i.text for i in driver.find_elements(By.CLASS_NAME, 'fts')[1:]]
    avgg = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.avg.bold')]
    
    
    lista_teams.append([team, link])
    
    
    table_data.append([team, mp, win, draw, loss, gf, ga, gd, pts, ppg, cs, btts, fts, avgg]) 
    table = {col : data for col, data in zip(colnames, table_data[0])}
    league_df = pd.DataFrame(table)
    return league_df



In [43]:
# guardar tablas de las ligas para temporada 22/23
def save_table_pd_21_22(league_df:pd.DataFrame, league:str)->None:

    league_df.to_csv(f'{league}_league_21_22.csv', index=False)

In [44]:
#temporada 21/22
chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://footystats.org/")

leagues = ["Premier League", "Serie A", "Ligue 1", "La Liga", "Bundesliga"]

errores = 0
for l in leagues:
    try:
        league_df = team_data_21_22(driver, l)
        save_table_pd_21_22(league_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1
    
driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League 2023/24 Table & Stats | FootyStats
Page title: Premier League 2023/24 Table & Stats | FootyStats
Page title: Serie A 2023/24 Table & Stats | FootyStats
Page title: Serie A 2023/24 Table & Stats | FootyStats
Page title: Ligue 1 2023/24 Table & Stats | FootyStats
Page title: Ligue 1 2023/24 Table & Stats | FootyStats
Page title: La Liga 2023/24 Table & Stats | FootyStats
Page title: La Liga 2023/24 Table & Stats | FootyStats
Page title: Bundesliga 2023/24 Table & Stats | FootyStats
Page title: Bundesliga 2023/24 Table & Stats | FootyStats
Proceso finalizado en 0:15:36.835395 con 0 errores
